In [4]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import deepxde as dde
import itertools
from collections import OrderedDict
import plotly.graph_objects as go # for plotting the graph
import pandas as pd
import seaborn as sb
from torch.autograd import Variable
from torch.autograd._functions import tensor
from torch.utils import data
from torch.utils.data import TensorDataset
from scipy.signal import find_peaks
from deepxde.data.data import Tuple
from deepxde.nn import initializers
from deepxde.nn.pytorch.nn import NN
from deepxde.nn import activations
from deepxde import config
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler  # 标准化工具
from sklearn.metrics import mean_squared_error # 均方误差
from deepxde.nn.pytorch.fnn import FNN
from deepxde.nn.pytorch.nn import NN
from deepxde.nn import activations
from deepxde.data import Data
from deepxde.data import BatchSampler
from deepxde.nn.deeponet_strategy import (
    SingleOutputStrategy,
    IndependentStrategy,
    SplitBothStrategy,
    SplitBranchStrategy,
    SplitTrunkStrategy,
)
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# 'cuda' 这里如果没有指定具体的卡号,系统默认cuda:0
device = torch.device('cuda:0') 		# 使用2号卡
print(torch.cuda.get_device_name(0)) # 查看使用的设备名称

NVIDIA GeForce RTX 4090 D


In [5]:

#DNN
class FNN(NN):
    """Fully-connected neural network."""

    def __init__(self, layer_sizes, activation, kernel_initializer, regularization=None ):
        super().__init__()
        if isinstance(activation, list):
            if not (len(layer_sizes) - 1) == len(activation):
                raise ValueError(
                    "Total number of activation functions do not match with sum of hidden layers and output layer!"
                )
            self.activation = list(map(activations.get, activation))
        else:
            self.activation = activations.get(activation)
        initializer = initializers.get(kernel_initializer)
        initializer_zero = initializers.get("zeros")    
        self.regularizer = regularization  # =None

        self.linears = torch.nn.ModuleList()

        for i in range(1, len(layer_sizes)):
            self.linears.append(
                torch.nn.Linear(
                    layer_sizes[i - 1], layer_sizes[i], dtype=config.real(torch)
                )
            )
            initializer(self.linears[-1].weight)
            initializer_zero(self.linears[-1].bias)

    def forward(self, inputs):
        x = inputs
        if self._input_transform is not None:
            x = self._input_transform(x)
        
        for j, linear in enumerate(self.linears[:-1]):
            x = (
                self.activation[j0](linear(x))
                if isinstance(self.activation, list)
                else self.activation(linear(x))
            )
        x = self.linears[-1](x)
        if self._output_transform is not None:
            x = self._output_transform(inputs, x)
        return x

In [6]:
xa_scaler=MinMaxScaler(feature_range=(0,1))
xb_scaler=MinMaxScaler(feature_range=(0,1))

train = pd.read_csv("C:\\Users\\ASUS001\\Desktop\\本科毕设\\孙国栋毕设\\间隙预测电流_训练集.csv",encoding='gbk')
u_train = train[['Time','Gap']]
u_train=xa_scaler.fit_transform(u_train)
s_train = train[['Current density']]
s_train=xb_scaler.fit_transform(s_train)
u_train = np.array(u_train, dtype='float32')
s_train = np.array(s_train, dtype='float32')
print(u_train.shape)
print(s_train.shape)

test = pd.read_csv("C:\\Users\\ASUS001\\Desktop\\本科毕设\孙国栋毕设\\间隙预测电流_测试集.csv",encoding='gbk')
u_test = test[['Time','Gap']]
u_test=xa_scaler.transform(u_test)  
s_test = test[['Current density']]
s_test=xb_scaler.transform(s_test)
u_test = np.array(u_test, dtype='float32')
s_test = np.array(s_test, dtype='float32')
print(u_test.shape)
print(s_test.shape)

data_train = Tuple(u_train, s_train, u_test, s_test)

(6642, 2)
(6642, 1)
(246, 2)
(246, 1)


In [11]:
net = FNN(
    [2, 30, 30,30, 30,1],
    "gelu",
    "Glorot normal",
    regularization=["l2",1e-8]
)
ModelCheckpoint = dde.callbacks.ModelCheckpoint(filepath='F:\\datasets\\pulsedDBD\\DNN4PulsedDBD\\model\\test',verbose=1,save_better_only=True,period=100,monitor="test loss")

In [ ]:
model = dde.Model(data_train, net)
model.compile("adamw", lr=0.01,decay=["step",50000,0.5], metrics=["mean l2 relative error"])
losshistory, train_state = model.train(iterations=50000,batch_size=None,callbacks=[ModelCheckpoint])
dde.utils.plot_loss_history(losshistory,fname='F:\\datasets\\sinDBD-frequency\\MaxNe2V\\losshistory.png') #fname是损失下降图片的保存地址

Compiling model...
'compile' took 0.642453 s

Training model...

Step      Train loss    Test loss     Test metric   
0         [3.41e-01]    [3.41e-01]    [inf]         


F:\anaconda3\envs\python\lib\site-packages\deepxde\metrics.py:26: RuntimeWarning: divide by zero encountered in divide
  np.linalg.norm(y_true - y_pred, axis=1) / np.linalg.norm(y_true, axis=1)


Epoch 100: train loss improved from inf to 3.41e-01, saving model to F:\datasets\pulsedDBD\DNN4PulsedDBD\model\test-100.pt ...

1000      [1.67e-03]    [1.67e-03]    [inf]         
Epoch 1000: train loss improved from 3.41e-01 to 1.67e-03, saving model to F:\datasets\pulsedDBD\DNN4PulsedDBD\model\test-1000.pt ...

2000      [3.52e-04]    [3.52e-04]    [inf]         
Epoch 2000: train loss improved from 1.67e-03 to 3.52e-04, saving model to F:\datasets\pulsedDBD\DNN4PulsedDBD\model\test-2000.pt ...

3000      [2.20e-04]    [2.20e-04]    [inf]         
Epoch 3000: train loss improved from 3.52e-04 to 2.20e-04, saving model to F:\datasets\pulsedDBD\DNN4PulsedDBD\model\test-3000.pt ...

4000      [1.78e-04]    [1.78e-04]    [inf]         
Epoch 4000: train loss improved from 2.20e-04 to 1.78e-04, saving model to F:\datasets\pulsedDBD\DNN4PulsedDBD\model\test-4000.pt ...

5000      [1.47e-04]    [1.47e-04]    [inf]         
Epoch 5000: train loss improved from 1.78e-04 to 1.47e-04, saving m

In [12]:
model = dde.Model(data_train, net)
model.compile("adamw", lr=0.0005,decay=["step",500000,0.5], metrics=["mean l2 relative error"])
model.restore('F:\\datasets\\pulsedDBD\\DNN4PulsedDBD\\model\\test-12000.pt')

Compiling model...
'compile' took 0.000258 s



RuntimeError: Error(s) in loading state_dict for FNN:
	size mismatch for linears.0.weight: copying a param with shape torch.Size([30, 1]) from checkpoint, the shape in current model is torch.Size([30, 2]).

In [ ]:
y_pred = model.predict(u_test)

plt.figure(figsize=(12, 5))
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plt.plot(xb_scaler.inverse_transform(s_test), label='True Output', linestyle='--')
plt.plot(xb_scaler.inverse_transform(y_pred), label='Predicted Output', alpha=0.7)

plt.xlabel('Time Step')
plt.ylabel('Current density')
plt.legend()

plt.show()